## 04_Задание Pivot

### Задача №1

In [1]:
#импортируем библиотеки для чтения предоставленных файлов:
import pandas as pd
import pathlib

In [2]:
#задаём путь к нашим файлам
path_1 = pathlib.Path('C:/') / 'Ya' / 'datas' / 'Отчет №1.xlsx' 
df_1 = pd.read_excel(path_1)
path_2 = pathlib.Path('C:/') / 'Ya' / 'datas' / 'Отчет №2.xlsx' 
df_2 = pd.read_excel(path_2)
path_3 = pathlib.Path('C:/') / 'Ya' / 'datas' / 'Отчет №3.xlsx' 
df_3 = pd.read_excel(path_3)

#выводим на экран первые 5 строк имеющихся данных для ознакомления
display(df_1.head())
display(df_2.head())
display(df_3.head())

,msisdn,call_status,result,call_start_time,duration,attempt
0,9733849194,486 busy here,NaN,2021-11-03 10:07:22,0,1
1,9710605134,486 busy here,NaN,2021-11-03 10:07:27,0,1
2,9856273132,200 OK,неудобно говорить после приветствия,2021-11-03 10:07:28,7,1
3,9797603702,200 OK,неудобно говорить после приветствия,2021-11-03 10:07:36,10,1
4,9285894273,486 busy here,NaN,2021-11-03 10:07:50,0,1


,msisdn,call_status,result,call_start_time,duration,attempt
0,9964074519,403 forbidden,NaN,2021-11-03 16:43:06,0,1
1,9358088939,486 busy here,NaN,2021-11-03 16:43:08,0,1
2,9603484358,200 OK,неудобно говорить после приветствия,2021-11-03 16:43:10,8,1
3,9700099386,200 OK,сброс на приветствии,2021-11-03 16:43:10,4,1
4,9095765474,403 forbidden,NaN,2021-11-03 16:43:11,0,1


,msisdn,call_status,result,call_start_time,duration,attempt
0,9060838584,486 busy here,NaN,2021-11-03 16:44:05,0,1
1,9426363077,486 busy here,NaN,2021-11-03 16:44:10,0,1
2,9071069561,486 busy here,NaN,2021-11-03 16:44:10,0,1
3,9003879782,486 busy here,NaN,2021-11-03 16:44:12,0,1
4,9700922955,486 busy here,NaN,2021-11-03 16:44:13,0,1


In [3]:
#добавляем метку с номером отчёта для дальнейшего объединения
df_1.insert(6, '#', 'Отчет №1', allow_duplicates = False)
df_2.insert(6, '#', 'Отчет №2', allow_duplicates = False)
df_3.insert(6, '#', 'Отчет №3', allow_duplicates = False)

# объединим имеющиеся данные
df = pd.concat([df_1, df_2, df_3])

#заменим отсутствующие значения на "пусто"
df.loc[df['result'].isna(),'result'] = 'пусто'

#столбец с датой преобразуем в удобный формат
df['call_start_time'] = pd.to_datetime(df['call_start_time'], format='%d%b%Y:%H:%M:%S.%f')

#создаём необходимые срезы согласно исходным данным
df_for_dozvon = df.query('result not in ["тишина", "автоответчик", "пусто"] and duration not in [0, 1, 2, 3]')
df_for_cr = df.query('result == "согласие"')

#создаём промежуточные сводные таблицы
pv_1 = df.groupby('#').agg({'msisdn':'nunique', 'call_start_time':'min', 'duration':'sum'})
pv_2 = df.groupby('#').agg({'call_start_time':'max'})
pv_3 = df_for_dozvon.groupby('#').agg({'msisdn':'nunique'})
pv_4 = df_for_cr.groupby('#').agg({'msisdn':'nunique'})

#объединяем промежуточные сводные таблицы
pv_final = pv_1.merge(pv_2, on='#', how='left').merge(pv_3, on='#', how='left').merge(pv_4, on='#', how='left')

#производим финальные расчёты и обогащение данных
pv_final['Дозвон,%'] = pv_final['msisdn_y'] / pv_final['msisdn_x'] * 100
pv_final['Конверсия,%'] = round(pv_final['msisdn'] / pv_final['msisdn_x'] * 100, 2)
pv_final['Минуты'] = round(pv_final['duration'] / 60)
pv_final = pv_final.rename(columns={'msisdn_x':'База', 'call_start_time_x':'start_time', 'call_start_time_y':'end_time'})
pv_final = pv_final.drop(['duration', 'msisdn_y', 'msisdn'], axis=1)
pv_final

,База,start_time,end_time,"Дозвон,%","Конверсия,%",Минуты
#,,,,,,
Отчет №1,2000,2021-11-03 10:07:22,2021-11-03 11:42:04,23.6,0.65,382.0
Отчет №2,1000,2021-11-03 16:43:06,2021-11-04 08:00:35,14.3,1.00,144.0
Отчет №3,600,2021-11-03 16:44:05,2021-11-03 16:46:26,30.0,6.67,219.0


### Задача №2

In [4]:
#заменим отсутствующие значения на "пусто"
df_1.loc[df_1['result'].isna(),'result'] = 'пусто'
df_2.loc[df_2['result'].isna(),'result'] = 'пусто'
df_3.loc[df_3['result'].isna(),'result'] = 'пусто'

#создадим промежуточные таблицы
pivot_1 = df_1.groupby(['call_status', 'result']).agg({'msisdn':'nunique', 'duration':'median'})
pivot_2 = df_2.groupby(['call_status', 'result']).agg({'msisdn':'nunique', 'duration':'median'})
pivot_3 = df_3.groupby(['call_status', 'result']).agg({'msisdn':'nunique', 'duration':'median'})

#обогащаем данные для дальнейших расчетов
pivot_1['Доля_1 от кол-ва уникальных номеров,%'] = (pivot_1['msisdn'] / df_1['msisdn'].nunique()) * 100
pivot_2['Доля_2 от кол-ва уникальных номеров,%'] = (pivot_2['msisdn'] / df_2['msisdn'].nunique()) * 100
pivot_3['Доля_3 от кол-ва уникальных номеров,%'] = (pivot_3['msisdn'] / df_3['msisdn'].nunique()) * 100

#объединяем полученные промежуточные таблицы в итоговую
pivot_final = (
    pivot_1.merge(pivot_2, on=['call_status','result'], how='outer')
    .merge(pivot_3, on=['call_status','result'], how='outer')
)

#переименуем столбцы чтобы названия соответствовали заданию
pivot_final = pivot_final.rename(columns = {'msisdn_x':'Отчёт №1. Число элементов в столбце msisdn',
                                            'duration_x':'Среднее время (медиана_1)',
                                            'msisdn_y':'Отчёт №2. Число элементов в столбце msisdn',
                                            'duration_y':'Среднее время (медиана_2)',
                                            'msisdn':'Отчёт №3. Число элементов в столбце msisdn',
                                            'duration':'Среднее время (медиана_3)'
                                           })

#для наглядности, отсортируем данные по убыванию среднего времени первого отчета 
pivot_final = pivot_final.sort_values(by='Среднее время (медиана_1)', ascending=False)
pivot_final

Отчёт №1. Число элементов в столбце msisdn  \
call_status                 result                                                                             
200 OK                      согласие                                                                    13.0   
                            отказ - вопрос по тарифу                                                     3.0   
                            отказ - 3 отказа                                                            73.0   
                            отказ - вопрос на какой номер услуга                                         1.0   
                            отказ - подключит самостоятельно                                             1.0   
                            уже пользуется тарифом                                                       3.0   
                            отказ - переход к другому оператору                                          4.0   
                            отказ - вопрос как подключить                                                2.0   
                            отказ - вопрос про срок                                                      1.0   
                            неявное согласие                                                             1.0   
                            отказ - вопрос сколько стоит                                                 1.0   
                            проблемы с распознаванием                                                    5.0   
                            отказ - дефолтное возражение                                                 1.0   
                            пожилой человек                                                              2.0   
                            отказ - нет необходимости                                                   74.0   
                            отказ - вопрос сколько сейчас плачу                                          1.0   
                            отказ - нет                                                                 21.0   
                            отказ - вопрос про копилку                                                   1.0   
                            отказ - подумает                                                             3.0   
                            отказ - хватает пакетов                                                     27.0   
                            отказ - дорого                                                               1.0   
                            отказ - просьба не менять                                                    3.0   
                            отказ - не пользуется                                                        1.0   
                            отказ                                                                        9.0   
                            сброс на предложении                                                        68.0   
                            больше не звонить                                                            3.0   
                            отказ от телемаркетинга                                                      2.0   
                            автоответчик                                                                 2.0   
                            неудобно говорить после приветствия                                         87.0   
                            тишина                                                                      68.0   
                            сброс на приветствии                                                        46.0   
                            сброс после приветствия                                                     23.0   
                            удобно говорить после приветствия                                            4.0   
486 busy here               пусто                                                                     1257.0   
481                         пусто 